In [2]:
import numpy as np 
import pandas as pd

In [19]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv('iris.csv', skiprows=1, header=None, names=col_names).sample(frac=1)
data["type"] = data["type"].replace(to_replace=["Setosa", "Versicolor", "Virginica"], value=[0, 1, 2])
data

,sepal_length,sepal_width,petal_length,petal_width,type
13,4.3,3.0,1.1,0.1,0
116,6.5,3.0,5.5,1.8,2
105,7.6,3.0,6.6,2.1,2
43,5.0,3.5,1.6,0.6,0
121,5.6,2.8,4.9,2.0,2
...,...,...,...,...,...
44,5.1,3.8,1.9,0.4,0
1,4.9,3.0,1.4,0.2,0
21,5.1,3.7,1.5,0.4,0
141,6.9,3.1,5.1,2.3,2


In [21]:
class Node():
    def __init__(self, feature_index = None, threshold=None, left=None, right=None, info_gain=None, value=None) -> None:
        
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        self.value = value

In [29]:
class DesicionTreeClassifier():
    def __init__(self, min_sample_split=2, max_depth=2):
        
        self.root = None
        
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        X, Y = dataset[:,:-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)
        
        if num_samples>=self.min_sample_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split['dataset_left'], curr_depth+1)
                right_subtree = self.build_tree(best_split['dataset_right'], curr_depth+1)
                
                return Node(best_split['feature_index'], best_split['threshold'],
                            left_subtree, right_subtree, best_split['info_gain'])
        
        leaf_value=self.calculate_leaf_value(Y)
        
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        
        best_split= {}
        max_info_gain =-float('inf')
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            
            for threshold in possible_thresholds:
                
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain( y, left_y, right_y, 'gini')
                    
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        return best_split
    
    
    def split(self, dataset, feature_index, threshold):
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        
        return dataset_left, dataset_right 

                                   
    def gini_index(self, y):
        
        class_lables = np.unique(y)
        gini = 0 
        for cls in class_lables:
            p_cls = len(y[y==cls])/len(y)
            gini += p_cls**2
        return 1-gini   
    
    def entropy(self, y):
        
        class_lables = np.unique(y)
        entropy = 0
        for cls in class_lables: 
            p_cls = len(y[y==cls])/len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy

                                                 
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain =  self.gini_index(parent) - (weight_l*self.gini_index(l_child)+ weight_r*self.gini_index(r_child)) 
        else:
            gain =  self.entropy(parent) - (weight_l*self.entropy(l_child)+ weight_r*self.entropy(r_child)) 
        
        return gain
    
    
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    
    def make_prediction(self, x, tree):
        
        if tree.value is not None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
        
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
            

In [30]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1, 1)

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)

In [31]:
classifier = DesicionTreeClassifier(min_sample_split=3, max_depth=3)
classifier.fit(X_train, Y_train)

In [32]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.9333333333333333